In [ ]:

from psutil import *

In [ ]:
cpu_count()  


In [ ]:
!lscpu |grep 'Model name'


In [ ]:
!df -h / | awk '{print $4}'

In [ ]:
!free -h --si | awk  '/Mem:/{print $2}'

In [ ]:
!nvidia-smi -L


In [ ]:
import random
from matplotlib import pyplot
from matplotlib.image import imread

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!unzip gdrive/MyDrive/DOG_CAT.zip

Archive:  gdrive/MyDrive/DOG_CAT.zip
   creating: DOG_CAT/
  inflating: DOG_CAT/Cat 0.jpg       
  inflating: DOG_CAT/Cat 1.jpg       
  inflating: DOG_CAT/Cat 10.jpg      
  inflating: DOG_CAT/Cat 100.jpg     
  inflating: DOG_CAT/Cat 101.jpg     
  inflating: DOG_CAT/Cat 102.jpg     
  inflating: DOG_CAT/Cat 103.jpg     
  inflating: DOG_CAT/Cat 104.jpg     
  inflating: DOG_CAT/Cat 105.jpg     
  inflating: DOG_CAT/Cat 106.jpg     
  inflating: DOG_CAT/Cat 107.jpg     
  inflating: DOG_CAT/Cat 108.jpg     
  inflating: DOG_CAT/Cat 109.jpg     
  inflating: DOG_CAT/Cat 11.jpg      
  inflating: DOG_CAT/Cat 110.jpg     
  inflating: DOG_CAT/Cat 111.jpg     
  inflating: DOG_CAT/Cat 112.jpg     
  inflating: DOG_CAT/Cat 113.jpg     
  inflating: DOG_CAT/Cat 114.jpg     
  inflating: DOG_CAT/Cat 115.jpg     
  inflating: DOG_CAT/Cat 116.jpg     
  inflating: DOG_CAT/Cat 117.jpg     
  inflating: DOG_CAT/Cat 118.jpg     
  inflating: DOG_CAT/Cat 119.jpg     
  inflating: DOG_CAT/Cat 12.j

In [ ]:
import os
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
# define location of dataset
folder = 'DOG_CAT/'
photos, labels = list(), list()
# enumerate files in the directory
for file in listdir(folder):
	# determine class
	output = 0.0
	if file.startswith('Dog'):
		output = 1.0
	# load image
	photo = load_img(folder + file, target_size=(200, 200))
	# convert to numpy array
	photo = img_to_array(photo)
	# store
	photos.append(photo)
	labels.append(output)
# convert to a numpy arrays
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)
# save the reshaped photos
save('dogs_vs_cats_photos.npy', photos)
save('dogs_vs_cats_labels.npy', labels)

(1008, 200, 200, 3) (1008,)


In [ ]:

# create directories
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['dogs/', 'cats/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		os.makedirs(newdir, exist_ok=True)

In [ ]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import shutil


In [ ]:
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = 'DOG_CAT/'
for file in listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = 'train/'
	if random() < val_ratio:
		dst_dir = 'test/'
	if file.startswith('Cat'):
		dst = dataset_home + dst_dir + 'cats/'  + file
		copyfile(src, dst)
	elif file.startswith('Dog'):
		dst = dataset_home + dst_dir + 'dogs/'  + file
		shutil.copyfile(src, dst)

In [ ]:

# baseline model for the dogs vs cats dataset
import sys
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [ ]:
# vgg16 model used for transfer learning on the dogs and cats dataset



# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow_from_directory('dataset_dogs_vs_cats/train/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	test_it = datagen.flow_from_directory('dataset_dogs_vs_cats/test/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)
	# evaluate model
	_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

58900480/58889256 [==============================] - 0s 0us/step
Found 779 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Found 229 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
13/13 [==============================] - 26s 849ms/step - loss: 1.1735 - accuracy: 0.9063 - val_loss: 0.6666 - val_accuracy: 0.9607
Epoch 2/10
13/13 [==============================] - 7s 501ms/step - loss: 0.1228 - accuracy: 0.9910 - val_loss: 0.6235 - val_accuracy: 0.9563
Epoch 3/10
13/13 [==============================] - 6s 529ms/step - loss: 0.0206 - accuracy: 0.9974 - val_loss: 0.5793 - val_accuracy: 0.9738
Epoch 4/10
13/13 [==============================] - 7s 525ms/step - loss: 0.0171 - accuracy: 0.9961 - val_loss: 0.5256 - val_accuracy: 0.9782
Epoch 5/10
13/13 [==============================] - 7s 530ms/step - loss: 0.0076 - accuracy: 0.9974 - val_loss: 0.6178 - val_accuracy: 0.9694
Epoch 6/10
13/13 [==============================] - 7s 503ms/step - loss: 0.0010 - accuracy: 0.9987 - val_loss: 0.7794 - val_accuracy: 0.9607
Epoch 7/10
13/13 [==============================] - 7s 509ms/step - loss: 0.0020 - accuracy: 0.9987 - val_loss: 0.5448 - val_accuracy: 0.9738
Epoch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.


> 96.943


In [ ]:
# organize dataset into a useful structure
# create directories
dataset_home_new = '/content/finalize_dogs_vs_cats/'
# create label subdirectories
labeldirs = ['dogs/', 'cats/']
for labldir in labeldirs:
	newdir = dataset_home_new + labldir
	makedirs(newdir, exist_ok=True)
# copy training dataset images into subdirectories
src_directory = '/content/dataset_dogs_vs_cats/train/'
for file in listdir(src_directory):
	src = src_directory  + '/' + file 
	if file.startswith('Cat'):
		dst = dataset_home_new + 'cats/'  + file
		shutil.copyfile(src, dst)
	elif file.startswith('Dog'):
		dst = dataset_home_new + 'dogs/'  + file
		shutil.copyfile(src, dst)

In [ ]:
# save the final model to file


# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

# run the test harness for evaluating a model
def run_test_harness():
# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow_from_directory('/content/dataset_dogs_vs_cats/train/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	# fit model
	model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=10, verbose=0)
	# save model
	model.save('final_model.h5')

# entry point, run the test harness
run_test_harness()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Found 779 images belonging to 2 classes.


In [ ]:
# make a prediction for a new image.
from keras.models import load_model
import matplotlib.pyplot as plt
from keras.preprocessing.image import array_to_img

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img



In [ ]:
from keras.preprocessing.image import save_img

def plot_figure(filename):
  # load the image
  img = load_img(filename)
  img_array = img_to_array(img)
  save_img( 'tahmin.jpg', img_array)
  
# show the image
  img.show()

In [ ]:
def plotting(filename,prediction):
	# plot loss
  pyplot.title(prediction)
# save plot to file
  img = load_img(filename)
  img_array = img_to_array(img) 
  filename = sys.argv[0].split('/')[-1]
  new_image = save_img('tahmin_edilen.jpg', img_array)
  imgplot = plt.imshow(img)
  plt.show
  pyplot.savefig(new_image + 'tahmin_edilen.jpg')
  pyplot.close()

In [ ]:
# load an image and predict the class
from IPython.display import Image
from keras.preprocessing import image
def run_example(image_file):
	# load the image
  img = load_image(image_file)
	# load model
  model = load_model('final_model.h5')
	# predict the class
  result = model.predict(img) 
  if result[0] <= 0.9:
    #title = image_file + '' + ' BU BİR KEDİ'
    title = image_file + '' + ' BU BİR KEDİ'
    print('Bu bir kedi')
  else: 
    #title= image_file + '' + ' BU BİR KÖPEK'
    title= image_file + '' + ' BU BİR KÖPEK'
    print('bu bir köpek')
  ###
  #plot_figure(image_file)
  #print(result[0])
  #plotting(image_file,title)
# entry point, run the example

#### TAHMİN EDİLECEK GÖRÜNTÜ ####
run_example('/content/gdrive/MyDrive/köpekcik.jpg')




OSError: ignored